# Pitch prediction

If a batter knows what pitch is coming, he is more likely to be able to hit it or lay off of it. Therefore, if a pitcher's next pitch can be predicted, a massive advantage goes to the batter. I will look into the possibility of pitch prediction in the following notebook.

I will start with importing some standard libraries.

In [1]:
# imports
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
import subprocess
%matplotlib inline

## Approach

I will look to predict pitches by using as few features as possible. Thus, I will begin by selecting raw features in the database based on human intuition. I intend to only turn to feature engineering if results are poor.

I intend to predict pitches using random forest. This approach has the advantages that it can display both low bias and low variance if results from a large number of decision trees are averaged. Additionally, random forest has a track record of good performance, can handle both regression and classification problems, and can also rank features based on influence on the response. Some drawbacks are that this approach can take up a lot of memory if the size of the dataset and the number of trees is large, and the forests themselves are a bit of a black box in terms of interpretability. Additionally, each pitcher is treated individually since each pitcher will have different pitch tendencies.

For the particular problem of pitch prediction, there are two possible responses that could be used: a categorical response that describes the pitch type (e.g., four-seam fastball, curveball, slider, etc.), or a numerical response that describes the trajectory of the ball (e.g., velocity and vertical/horizontal acceleration).

Ideally, there would be reliable pitch-type labels so that noise in the response that's being predicted would be limited. For example, noise from slightly different trajectories of the same pitch would be limited by classifying them as the same pitch. Additionally, this approach would have a simple baseline metric from which to test my model against: does the model perform better than guessing the most commonly thrown pitch type every pitch? However, from previous work with pitch classification, I concluded that all pitch-type labels should be taken with caution. For instance, the difference between a two-seam and a four-seam fastball can be subjective, and the number of pitches in a pitcher's repertoire is also subjective. One option is to group all pitches into two labels: fastball and off-speed. While this approach would limit the number of misclassified pitches, it also fails to fully exploit the wealth of pitch trajectory information in the database.

Alternatively, I could fully utilize the information in the database and try to predict the trajectory of the next pitch. From my pitch classification work, it appeared that velocity and acceleration were the most important features for separating pitch type, making them candidates for prediction. This approach, however, would be sensitive to noise in the data. Again, a given pitch type is not executed perfectly every time, resulting in a range of trajectories. I am not interested so much in fitting the exact pitch trajectory of the next pitch as I am in estimating an approximate pitch trajectory that allows the hitter to distinguish between pitch types. This approach has the additional challenge that it is difficult to specify a baseline metric to test my model against. Would I just guess the average trajectory of all pitch types every pitch?

Therefore, I propose to use a hybrid approach of the two potential responses: predict how the next pitch falls into bins of velocities and accelerations. So instead of pitches being labeled by pitch type, they will have three labels related to velocity and acceleration bins. Each label type (velocity, vertical and horizontal acceleration) will be split into bins (or categories). For instance, velocity could be split into bins of 70-, 70-80, 80-90, and 90+ mph (these could be different values depending on the pitcher). By predicting which bin the next pitch falls into for each of the three categories (independently), I am able to utilize the pitch trajectory information directly while limiting the effect of noise in the data at the same time. I would no longer be fitting subjective pitch type labels. For this particular approach, the baseline metric to test my model against could be guessing that every pitch is the bin with the highest pitch density.

## Loading data

Pitch prediction will not only depend on the previous pitch thrown, but also variables such as game situation. Therefore, I will load the *pitchfx*, *events*, and *games* tables in the database as pandas dataframes so that I maintain maximum flexibility. I will focus on Barry Zito again for this study.

In [5]:
# specify database name
dbname = "../dat/pitchfx2008.db"

# connect to the sqlite3 database
db = sqlite3.connect(dbname)
hdb = db.cursor()

# import classes
sys.path.append('../src')
from Player import Player

# create player object
bz = Player("Barry Zito")

# grab pitches
bz_pitches = bz.pitches(db, clean=1)
# grab events
bz_events = bz.pitch_events(db)
# grab games
bz_games = bz.pitch_games(db)

## Data preprocessing

Feature selection can sometimes be more art than science. Here, I will prepare a subset of features that intuition suggests will help predict the type of pitch thrown. They are, loosely, the current ball-strike count, the handedness of the batter, the baserunner situation, and the previous pitch thrown. I will prepare these features one by one.

### Ball-strike count

Pitchers will likely pitch differently depending on the ball-strike count. For instance, if the count is 0-2 (0 balls, 2 strikes), a pitcher can throw any pitch because he knows he has 3 more tries to get the batter out. On the other hand, if the count is 3-0 (3 balls, 0 strikes), a pitcher will likely throw a pitch he trusts most in terms of command (often a fastball) because the pitcher will oftentimes want to avoid walking the batter.

In terms of how to represent the count as a feature, there are a couple of options: treat the number of balls and the number of strikes independently, or treat each count as its own category. For now, I will treat the number of balls and strikes as independent features because it requires less feature manipulation. Furthermore, I will keep their original values (0-3 for balls, 0-2 for strikes) rather than encode them, as the values of these numbers have meaning.

In [11]:
# create first feature
features = bz_pitches[["pre_balls", "pre_strike"]]

### Handedness of batter

The next pitch is also likely influenced by the handedness of the batter. For example, certain pitches are more effective when the pitcher handedness and batter handedness match (such as a breaking ball), and are thus more likely to be thrown.

To get the handedness of the batter for each pitch, I'll need to join the *pitchfx* table with both the *events* table (to tie the pitch to a batter id) and the *players* table (to tie the batter id to the batter handedness). I will do this all in pandas, which requires me to import the *players* table as a pandas dataframe.

In [13]:
# save players table as pandas data frame
query = """SELECT *
        FROM players
        """
players = pd.read_sql_query(query, db)

In [30]:
# merge tables to get the batter handedness for each pitch
#handedness = bz_pitches.merge(bz_events, on=['game_id', 'event_id'])
handedness = bz_pitches.merge(bz_events[['game_id', 'event_id', 'batter_id']],
                              left_on=['game_id', 'prev_event'],
                              right_on=['game_id', 'event_id'])

In [31]:
handedness

,game_id,pitch_num,at_bat,time,prev_event,description,outcome,pre_balls,post_balls,pre_strike,...,ay,az,break_y,break_angle,break_length,spin_dir,spin_rate,pitch_type,event_id,batter_id
0,233769,30,4,134206.0,7,Ball,B,0,1,0,...,28.973,-13.125,23.7,16.5,3.9,192.990,2278.601,FC,7,279577
1,233769,31,4,134217.0,7,Called Strike,S,1,1,0,...,29.654,-9.818,23.7,8.9,3.2,184.540,2620.543,FF,7,279577
2,233769,32,4,134228.0,7,Foul,S,1,1,1,...,22.606,-23.886,23.7,-8.3,9.2,156.564,1243.876,CH,7,279577
3,233769,33,4,134247.0,7,"In play, no out",X,1,1,2,...,30.550,-8.798,23.7,12.1,2.8,183.501,2732.740,FF,7,279577
4,233769,37,5,134336.0,8,Ball,B,0,1,0,...,31.828,-12.756,23.6,18.5,3.8,190.573,2300.700,FF,8,279577
5,233769,38,5,134356.0,8,Foul,S,1,1,0,...,31.309,-9.620,23.7,-0.4,3.0,177.663,2608.546,FF,8,279577
6,233769,39,5,134425.0,8,"In play, out(s)",X,1,1,1,...,25.072,-25.642,23.6,-11.1,9.4,139.587,1146.969,CH,8,279577
7,233769,43,6,134519.0,9,Called Strike,S,0,0,0,...,29.577,-6.778,23.7,9.5,2.3,184.039,2963.215,FF,9,444843
8,233769,44,6,134539.0,9,"In play, run(s)",X,0,0,1,...,22.197,-20.559,23.7,-9.6,7.5,162.426,1626.510,CH,9,444843
9,233769,51,7,134641.0,10,Ball,B,0,1,0,...,27.938,-12.132,23.7,-5.0,3.3,174.978,2316.492,FF,10,461314


In [36]:
bz_pitches.head()

,game_id,pitch_num,at_bat,time,prev_event,description,outcome,pre_balls,post_balls,pre_strike,...,vz0,ax,ay,az,break_y,break_angle,break_length,spin_dir,spin_rate,pitch_type
0,233769,30,4,134206.0,7,Ball,B,0,1,0,...,-4.167,-4.395,28.973,-13.125,23.7,16.5,3.9,192.990,2278.601,FC
1,233769,31,4,134217.0,7,Called Strike,S,1,1,0,...,-7.852,-1.776,29.654,-9.818,23.7,8.9,3.2,184.540,2620.543,FF
2,233769,32,4,134228.0,7,Foul,S,1,1,1,...,-2.479,3.593,22.606,-23.886,23.7,-8.3,9.2,156.564,1243.876,CH
3,233769,33,4,134247.0,7,"In play, no out",X,1,1,2,...,-5.803,-1.431,30.550,-8.798,23.7,12.1,2.8,183.501,2732.740,FF
4,233769,37,5,134336.0,8,Ball,B,0,1,0,...,-2.421,-3.625,31.828,-12.756,23.6,18.5,3.8,190.573,2300.700,FF


In [34]:
bz_events

,game_id,event_id,event_description,inning,is_top_inning,pre_out,post_out,pitcher_id,batter_id,runner_id,run_start,run_end,home_score,away_score
0,233769,7,-1,1,0,0,0,217096,279577,-1,-1,-1,0,0
1,233769,8,Double,1,0,0,0,217096,279577,279577,H,2B,0,0
2,233769,9,Pop Out,1,0,0,1,217096,444843,444843,H,0,0,0
3,233769,10,Single,1,0,1,1,217096,461314,279577,2B,H,1,0
4,233769,10,Single,1,0,1,1,217096,461314,461314,H,2B,1,0
5,233769,11,Pickoff Attempt 2B,1,0,1,1,217096,116999,461314,H,H,3,0
6,233769,12,Pickoff Attempt 2B,1,0,1,1,217096,116999,116999,H,H,3,0
7,233769,12,Pickoff Attempt 2B,1,0,1,1,217096,116999,461314,2B,H,3,0
8,233769,13,Groundout,1,0,1,2,217096,116662,116662,H,0,3,0
9,233769,14,Flyout,1,0,2,3,217096,431145,431145,H,0,3,0
